# How to use Hive in a Jupyter notebook

* [Create a Hive kernel with the ppextensions](#create)
* [Open a Hive kernel](#open)
* [Load extension and set connection to the hive metastore](#load)
* [Use usual hive query commands with the magic command `%hive`](#use)

### Create a Hive kernel with the ppextensions<a class="anchor" id="create"></a>

1. (that step might be optional) Change/check on Ambari that the hiveserver2 transport is in binary mode: the parameter hive.server2.transport.mode must be set to binary (and not http). Hive listens on port 10000 in binary and on port 10001 in http mode. Check that the machine hosting the hiveserver2 is listening to the port 10000 <br>
&ensp;&ensp;&ensp;&ensp;`$ netstat -an | grep 10000`<br>
&ensp;&ensp;&ensp;&ensp;`tcp        0      0 0.0.0.0:10000           0.0.0.0:*               LISTEN`
<br>

1. Connect to the Jupyter machine as the user that will use hive in Jupyter <br>
&ensp;&ensp;&ensp;&ensp;`$ su myuser`
<br>

1. Add environment variables with anaconda directories: in the .bashrc (create if does not exist) add the lines: <br>
&ensp;&ensp;&ensp;&ensp;`export CONDA_ENVS_PATH=/path/to/condaenvs/myuser/envs`<br>
&ensp;&ensp;&ensp;&ensp;`export CONDA_PKGS_DIRS=/path/to/condaenvs/myuser/pkgs`<br>
&ensp;&ensp;&ensp;&ensp;`export PATH=/path/to/anaconda3-4.3.1/bin/:$PATH`
<br>

1. Open a new terminal or source the .bashrc<br>
&ensp;&ensp;&ensp;&ensp;`$ source /home/myuser/.bashrc`
<br>

1. Give permission to usertest to use the default queue: if we are connected as usertest and execute <br>
&ensp;&ensp;&ensp;&ensp;`$ mapred queue -showacls`<br>
we must see `SUBMIT_APPLICATIONS` for the default queue.
<br>

1. Create directories for environments and packages for usertest in the anaconda install.<br>
&ensp;&ensp;&ensp;&ensp;`$ cd /path/to/condaenvs/`<br>
&ensp;&ensp;&ensp;&ensp;`$ mkdir -p myuser/envs myuser/pkgs`
<br>

1. Create a conda environment and enter in the environment.<br>
&ensp;&ensp;&ensp;&ensp;`$ conda create -n hive_kernel`<br>
&ensp;&ensp;&ensp;&ensp;`$ source activate hive_kernel`
<br>
	
1. Check that the conda command points to conda of the environment. <br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ which conda`<br>
&ensp;&ensp;&ensp;&ensp;`/path/to/condaenvs/myuser/envs/hive_kernel/bin/conda`
<br>

1. Install pip in the environment and check that the command points to the bin of the environment.<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ conda install pip`<br>
&ensp;&ensp;&ensp;&ensp;`...`<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ which pip`<br>
&ensp;&ensp;&ensp;&ensp;`/path/to/condaenvs/myuser/envs/hive_kernel/bin/pip`
<br>

1. We are going to use pip to install all that we need. If we install several packages with both pip and conda we are going to have compatibility / package management / dependency problems. We only need to install the ppextensions packages (the dependencies will be installed automatically)<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ pip install ppextensions`<br>
In the successfully installed packages we see ipython ipykernel jupyter-console jupyter-core among others, which are packages needed to run the environment on jupyter. We DO NOT (try to) install them again with pip or conda.
<br>

1. Check that the command ipython points to the bin of the environment and use it to install the environment as a kernel for jupyter for the current user. The environment will be found in jupyter as the kernel named Hive.<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ which ipython`<br>
&ensp;&ensp;&ensp;&ensp;`/hadoop/lab/condaenvs/usertest/envs/hive_kernel/bin/ipython`<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ /hadoop/lab/condaenvs/usertest/envs/hive_kernel/bin/ipython kernel install --user --name Hive`
<br>


**TROUBLESHOOTING**

If you want to restart the procedure from the beginning (in case of errors), you need to get out of the environment, remove it, and manually remove some packages that are not removed automatically.<br>
&ensp;&ensp;&ensp;&ensp;`(hive_kernel) $ source deactivate`<br>
&ensp;&ensp;&ensp;&ensp;`$ conda remove --name hive_kernel --all`<br>
&ensp;&ensp;&ensp;&ensp;`$ conda env list`<br>
&ensp;&ensp;&ensp;&ensp;`--check that Hive is not in the list anymore--`<br>
&ensp;&ensp;&ensp;&ensp;`$ ls myuser/pkgs/`<br>
&ensp;&ensp;&ensp;&ensp;`$ rm -rf  myuser/pkgs/*`<br>


### Open a Hive kernel <a class="anchor" id="open"></a>

Connect to jupyter with usertest. Click new, a drop down menu appears with the new kernel HiveQl.

### Load extension and set connection to the hive metastore <a class="anchor" id="load"></a>

Load the package for the extensions.

In [1]:
%load_ext ppextensions.ppmagics

Load some configurations for hive. There should be a way to put all that in a conf file somewhere and not having to load it everytime we open a new notebook with the Hive kernel.
- `-hs` Machine that hosts the hiveserver2. Check on the host machine what is the form of the principal with "`klist -kt /etc/security/keytabs/hive.service.keytab`". If the Hive principal is `hive/machine.my.com@REALM` you need to include .my.com in the address of the `-hs` option, if it is not present don't put it in the `-hs` option (i.e. only machine).
- `-p`  Port of the hiveserver2 in binary mode (because we set the hiverser2 transport to binary at the beginning)
- `-a`  Authentication mode, `gssapi` for kerberos

In [2]:
%hive -hs machine.my.com -p 10000 -a gssapi

HTML(value='')

[]

### Use usual hive query commands with the magic command `%hive` <a class="anchor" id="use"></a>

In [4]:
%hive use default

HTML(value='')

[]

In [6]:
%hive select * from test limit 10

HTML(value='')

test.passengerid,test.pclass,test.name,test.sex,test.age,test.sibsp,test.parch,test.ticket,test.fare,test.cabin,test.embarked
PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin
1,0,3,"""Braund","Mr. Owen Harris""",male,22,1,0,A/5 21171,7.25
2,1,1,"""Cumings","Mrs. John Bradley (Florence Briggs Thayer)""",female,38,1,0,PC 17599,71.2833
3,1,3,"""Heikkinen","Miss. Laina""",female,26,0,0,STON/O2. 3101282,7.925
4,1,1,"""Futrelle","Mrs. Jacques Heath (Lily May Peel)""",female,35,1,0,113803,53.1
5,0,3,"""Allen","Mr. William Henry""",male,35,0,0,373450,8.05
6,0,3,"""Moran","Mr. James""",male,,0,0,330877,8.4583
7,0,1,"""McCarthy","Mr. Timothy J""",male,54,0,0,17463,51.8625
8,0,3,"""Palsson","Master. Gosta Leonard""",male,2,3,1,349909,21.075
9,1,3,"""Johnson","Mrs. Oscar W (Elisabeth Vilhelmina Berg)""",female,27,0,2,347742,11.1333
